In [2]:
import numpy as np
import scipy.io as io
import scipy.sparse as sparse

# Load a text file of integers:
y = np.loadtxt("upvote_labels.txt", dtype=np.int)

# Load a text file of strings:
featureNames = open("upvote_features.txt").read().splitlines()

# Load a csv of floats:
A = np.genfromtxt("upvote_data.csv", delimiter=",")

# Load a matrix market matrix, convert it to csc format:
B = io.mmread("star_data.mtx").tocsc()



In [12]:
# A[0:5,0:5]
y.shape
y[1:10]
print A.shape
print A[0:4,0:4]
print B.shape

(6000, 1000)
[[  1.83545800e-05   9.89344400e-03   0.00000000e+00   1.52754600e-03]
 [  3.67091600e-05   9.34219200e-03   0.00000000e+00   1.36206200e-03]
 [  0.00000000e+00   6.50506700e-03   0.00000000e+00   6.60393700e-04]
 [  1.07374300e-03   2.01565700e-02   0.00000000e+00   6.34063400e-03]]
(45000, 2500)


In [13]:
def myCooDescent(w0,w,y,X,lyambda):
    N,d=X.shape
#     print d,N
    wOld=w #keep for computations
    wNew=wOld 
    kmax=50 #max number of iterations
    eps=10**(-3) # for convergence
    w0old=w0
    
    for kk in range(kmax):
#         pdb.set_trace()
        errorOld=np.square(np.linalg.norm(y-X*wOld-w0old, ord=2))+lyambda*np.linalg.norm(wOld, ord=1)
        
        yhat=X*wOld+w0old
        w0new=np.sum(y-yhat+w0old)/N
        yhat=yhat-w0old+w0new
    
        for jj in range(d):
#             pdb.set_trace()
            start, end = X.indptr[jj], X.indptr[jj+1]
            aj=2*np.sum(np.square(X.data[start:end]))
            
            cj=2*np.sum(X.data[start:end]*(y[X.indices[start:end]]-\
                                               yhat[X.indices[start:end]]+X.data[start:end]*wOld[jj]))
            if cj+lyambda<0:
                wNew[jj]=(cj+lyambda)/aj
            elif cj-lyambda>0:
                wNew[jj]=(cj-lyambda)/aj
            else:
                wNew[jj]=0
                
        if np.linalg.norm(wNew-wOld,ord=np.inf)<eps:
#             print('converged')
            return (w0,wNew)
        else:
            wOld=wNew
            w0old=w0new
            
        errorNew=np.square(np.linalg.norm(y-X*wNew-w0, ord=2))+lyambda*np.linalg.norm(wNew, ord=1)
        if errorOld-errorNew<10**(-3):
            print('error grows', jj)
                
    if kk==kmax-1:
        print('reached max iter')
        return (w0, wNew)
            

In [ ]:
lyambda=2*np.linalg.norm(np.dot(XT,y-np.mean(y)), ord=np.inf)
nloop=5
# precision=np.zeros(nloop)
# recall=np.zeros(nloop)
for i in range(30):
    w0,wGuess=myCooDescent(w0,wGuess,y,Xsparse,lyambda)
    lyambda=0.9*lyambda
#     print(w0, wGuess[0:10])